In [1]:
import numpy as np
import sys
import logging
import concurrent.futures
import time
from datetime import datetime
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Transformer, CRS
import shapely.geometry as sg
from shapely.geometry import Polygon, Point
from sliderule import sliderule, icesat2, earthdata, h5, ipysliderule, io
import warnings
from IPython import display
import json
import utils.toolshelf as t
import LagrangianDEMdifferencing.utils.lagrange as lagrange
from random import sample

import fiona

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
####################### SET PARAMETERS #################

## Select predetermined track (2 digit cycle number)
site = "filchnerF1"
startCycle = '01'
endCycle = '22'

## Data access options

# (-1: Skip altogether, 0: process fresh, 1: load from geojson, 2: load from csv)
accessType03 = 1
accessType06 = 1

ignoreTracks=None

## input options

# resolution (recommended 20.0)
# "len" and "res" (will be equal)
res = 10.0
cnf = 2

## output options

# Save flag 06 (0: dont save, 1: save as geojson (slow but everything works), 
# 2: save as csv (faster but nothing works))
sFlag06 = 0

# Save flag melange (0: dont save, 1: save as geojson)
sFlagm = 0

# Save flag 03 (0: dont save, 1: save as geojson)
sFlag03 = 0

## Some defaults for file namingd
# _save and _load is to prevent accidentally overwriting good data
# as such, freshly created data needs to be managed manually
datRoot = f"../data/IS2/"
file03_save = f"{datRoot}{site}_ATL03_cyc{cycle}_res{int(res)}"
file06_save = f"{datRoot}{site}_ATL06_cyc{cycle}_res{int(res)}"
fileMel_save = f"{datRoot}{site}_mel_cyc{cycle}_res{int(res)}"
file03_load = f"{datRoot}{site}_res{int(res)}/{site}_ATL03_cyc{cycle}_res{int(res)}"
file06_load = f"{datRoot}{site}_res{int(res)}/{site}_ATL06_cyc{cycle}_res{int(res)}"
fileMel_load = f"{datRoot}{site}_mel_res{int(res)}/{site}_mel_cyc{cycle}_res{int(res)}"

In [ ]:
# Configure ICESat-2 API through SlideRule
icesat2.init("slideruleearth.io", verbose=False)

#Params
parms = {
    "poly": t.getRegion(site, cycle),
    "srt": 0,
    "len": res*2,
    "res": res,
    "cnf": cnf,
    "maxi": 6,
    "ats": 5.0,
    "cnt": 5,
    "H_min_win": 3.0,
    "sigma_r_max": 5.0,
    "cycle": cycle,
    "atl03_geo_fields": ['geoid', 'dem_h', 'dem_flag', 'tide_earth', 'tide_ocean', 'geoid_free2mean', 'tide_earth_free2mean']
}

In [ ]:
%%time
# ATL03 Processing

## add something to check if the data already exists

##### Set ATL03 sp parameters ##############################
parms = {
    "poly": region,
    "srt": 0,
    "len": res*2,
    "res": res,
    # identify ground track
    "track": tr,
    # classification and checks
    # still return photon segments that fail checks
    "pass_invalid": True,
    # all photons
    "cnf": 2,
    "cnt": 5,
    "atl03_geo_fields": ["ref_azimuth", "ref_elev", "geoid"],
    "atl03_ph_fields": ["delta_time"],
    # all land classification flags
    "atl08_class": ["atl08_noise", "atl08_ground", "atl08_canopy", "atl08_top_of_canopy", "atl08_unclassified"],
    # all photons
    #"yapc": dict(knn=0, win_h=6, win_x=11, min_ph=4, score=100), 
}

#Have a list of only one granule
gdf = icesat2.atl03sp(parms, resources=[trackID])
atl03_yapc = gdf[gdf.pair==pair]

# Display Statistics
t.display03Info(atl03_yapc)

if sFlag03==1:
    print(f'Saving file as geojson named {file03_save}.geojson')
    atl03_yapc.to_file(f"{file03_save}.geojson", driver='GeoJSON')